# Importing necessary libraries

In [ ]:
!pip install contractions colorama rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 7.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e868d61660118c18fb4d9f92c7060b91c403c5344c1671295ab4455f9b141a65
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
import contractions
import unicodedata
import chardet
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re
import networkx as nx
from rouge_score import rouge_scorer
import networkx as nx
from bs4 import BeautifulSoup

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#Load the dataset
dataset=pd.read_excel('366_ARPs_for_extracting_Issue_Solution_Pairs.xlsx')

# Preprocessing the Text

In [ ]:
# Applying heuristic technique to reduce noice in the data
def clean_dataset(text):
    if not isinstance(text, str):
        return text

    soup = BeautifulSoup(text, "html.parser")

    for a in soup.find_all('a'):
        a.replace_with('[external-link]')

    for img in soup.find_all('img'):
        img.replace_with('[figure]')

    for code in soup.find_all('code'):
        code.replace_with('[code-snippet]')

    for table in soup.find_all('table'):
        table.replace_with('[table]')

    clean_text = soup.get_text()

    return clean_text

# Apply the function to 'Question_body' and 'Answer_body' columns
dataset['Question_body_cleaned'] = dataset['Question_body'].apply(clean_dataset)
dataset['Answer_body_cleaned'] = dataset['Answer_body'].apply(clean_dataset)

In [ ]:
# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    sentences = sent_tokenize(text)
    stop_words = set(ENGLISH_STOP_WORDS)
    punctuation = set(string.punctuation)
    extra_special_characters = set(["''", '``', '##', '>>', '<<', 'e', 'g', 'eg', 'cant', 'cannot', 'isnt', 'would', 'could', 'doesnt', 'hasnt', 'thanks', '-', ')', '\n'])
    special_characters = extra_special_characters.union(punctuation)

    processed_sentences = []

    for sentence in sentences:
        cleaned_words = re.findall(r'\b\w+\b', sentence.lower())
        cleaned_words = [word for word in cleaned_words if word not in stop_words]
        cleaned_words = [word for word in cleaned_words if word not in special_characters]
        cleaned_words = [word for word in cleaned_words if not word.isdigit()]
        cleaned_words = [word for word in cleaned_words if word.isalpha()]
        lemmatized_words = [lemmatizer.lemmatize(word) for word in cleaned_words]

        processed_sentences.append(cleaned_words)

    return sentences, processed_sentences

In [ ]:
# Preprocess the Question_body and Answer_body columns
dataset['processed_question'] = dataset['Question_body_cleaned'].apply(preprocess_text)
dataset['processed_answer'] = dataset['Answer_body_cleaned'].apply(preprocess_text)

#dataset[['Question_title', 'processed_question', 'processed_answer']].head()

# Vectorization and Similarity Matrix Computation

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def build_similarity_matrix(processed_sentences):
    sentences = [" ".join(sentence) for sentence in processed_sentences]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    similarity_matrix = cosine_similarity(tfidf_matrix)

    return similarity_matrix

# Build similarity matrices for both processed questions and answers
dataset['question_similarity_matrix'] = dataset['processed_question'].apply(lambda x: build_similarity_matrix(x[1]))
dataset['answer_similarity_matrix'] = dataset['processed_answer'].apply(lambda x: build_similarity_matrix(x[1]))

dataset[['question_similarity_matrix', 'answer_similarity_matrix']].head()


,question_similarity_matrix,answer_similarity_matrix
0,"[[1.0, 0.0, 0.09473376964436592, 0.0, 0.0, 0.0...","[[1.0000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,"[[1.0000000000000002, 0.0, 0.0, 0.0, 0.0], [0....","[[1.0, 0.0, 0.0865651075502646, 0.034802321376..."
2,"[[1.0, 0.07028081026664326, 0.0690588812162663...","[[0.9999999999999999, 0.17557494911455973, 0.1..."
3,"[[0.9999999999999999, 0.0, 0.09738511301260477...","[[1.0000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,"[[0.9999999999999999, 0.097745674227839, 0.033...","[[1.0, 0.11062087717781531, 0.1267026034121509..."


# Graph Construction, Ranking with Pagerank:

In [ ]:
def text_rank_with_networkx(similarity_matrix, damping_factor=0.85):
    n = similarity_matrix.shape[0]
    # Create a directed graph
    graph = nx.DiGraph()

    for i in range(n):
        for j in range(n):
            if i != j and similarity_matrix[i, j] > 0:  # Avoid self-loops
                graph.add_edge(i, j, weight=similarity_matrix[i, j])

    # Compute PageRank
    scores = nx.pagerank(graph, alpha=damping_factor)
    return np.array(list(scores.values()))

dataset['question_scores'] = dataset['question_similarity_matrix'].apply(text_rank_with_networkx)
dataset['answer_scores'] = dataset['answer_similarity_matrix'].apply(text_rank_with_networkx)


# Generating the Summary/Issue-solution pairs

In [ ]:
def text_rank(similarity_matrix, damping_factor=0.85, max_iter=100, tol=1e-4):
    n = similarity_matrix.shape[0]
    scores = np.ones(n) / n
    for _ in range(max_iter):
        prev_scores = scores.copy()
        for i in range(n):
            scores[i] = (1 - damping_factor) + damping_factor * np.sum(similarity_matrix[i] * prev_scores / np.sum(similarity_matrix[i]))
        if np.linalg.norm(scores - prev_scores) < tol:
            break
    return scores

# Apply TextRank to the similarity matrices for both questions and answers
dataset['question_scores'] = dataset['question_similarity_matrix'].apply(text_rank)
dataset['answer_scores'] = dataset['answer_similarity_matrix'].apply(text_rank)

dataset[['question_scores', 'answer_scores']].head()

"""# Generating the Summary"""

def generate_summary(original_sentences, scores, num_sentences=5):
    ranked_sentence_indices = np.argsort(scores)[::-1]

    summary = []
    for i in range(min(num_sentences, len(original_sentences))):
        summary.append(original_sentences[ranked_sentence_indices[i]])

    return " ".join(summary)

# Generate issues and solutions for both questions and answers
dataset['Issue_Extracted'] = dataset.apply(lambda x: generate_summary(x['processed_question'][0], x['question_scores']), axis=1)
dataset['Solution_Extracted'] = dataset.apply(lambda x: generate_summary(x['processed_answer'][0], x['answer_scores']), axis=1)

<ipython-input-10-0b3d2d54b0ad>:7: RuntimeWarning: invalid value encountered in divide
  scores[i] = (1 - damping_factor) + damping_factor * np.sum(similarity_matrix[i] * prev_scores / np.sum(similarity_matrix[i]))


In [ ]:
import chardet

def detect_and_fix_encoding(text):
    if not isinstance(text, str):
        raise ValueError("Input must be a string")

    try:
        result = chardet.detect(text.encode('utf-8', errors='ignore'))
        encoding = result['encoding']

        if encoding:
            return text.encode(encoding, errors='ignore').decode('utf-8', errors='replace')
        else:
            return text
    except Exception as e:
        print(f"Error processing text: {e}")
        return text

dataset['Issue_Extracted'] = dataset['Issue_Extracted'].apply(detect_and_fix_encoding)
dataset['Solution_Extracted'] = dataset['Solution_Extracted'].apply(detect_and_fix_encoding)


In [ ]:
# Display the summaries/issue-solution pairs
summaries = dataset[['Question_title', 'Issue_Extracted', 'Solution_Extracted']]
summaries.head()

,Question_title,Issue_Extracted,Solution_Extracted
0,Separation of Students and Users in NestJS Mic...,"Create student, assign institute to the studen...","But still, if you inject repository for one en..."
1,Flutter Clean Architecture,I hope I explain my question clearly. I tried ...,"You can follow the architecture as, [code]. D..."
2,Correct .NET Architecture for long running asy...,I am storing the task details in a MS SQL Serv...,Cancellation of the operation can also be mark...
3,Architecture for white-label mobile apps with ...,"Application functionality may differ slightly,...",Use dependency injection -> Let's call a piece...
4,Implementing Data Source Selection Logic in Cl...,One solution seems to be writing a separate Us...,"The logic does not belong in the Repository, i..."


In [ ]:
from colorama import Fore, Style
import textwrap

def format_summary(row, width=80):
    title = f"{Fore.RED}Title:{Fore.WHITE} {row['Question_title']}"
    question_summary = f"{Fore.GREEN}Issue_Extracted:{Fore.WHITE}\n{textwrap.fill(row['Issue_Extracted'], width=width)}"
    answer_summary = f"{Fore.BLUE}Solution_Extracted:{Fore.WHITE}\n{textwrap.fill(row['Solution_Extracted'], width=width)}"

    return f"{title}\n\n{question_summary}\n\n{answer_summary}\n{Fore.YELLOW}{'-'*width}{Fore.WHITE}\n"

formatted_summaries = summaries.head(6).apply(format_summary, width=80, axis=1)

# Display the formatted issue-solutions
for summary in formatted_summaries:
    print(summary)

Title: Separation of Students and Users in NestJS Microservice architecture

Issue_Extracted:
Create student, assign institute to the student, insert documents, are part of
the student folder and repository, and entity files related to these are stored
in the student folder. Also, creating a student involves a step of creating a
user as well. I am still able to do stuff I need to do, I am just thinking, that
tomorrow If I switch to a microservices approach where the user and student will
be different microservices with different databases, I will not be able to
inject the repository and somehow need to call the Rest API for user or student
to achieve this. Considering this, creating users, contacts, and addresses are
part of the user folder, and repository files and entity files related to these
are stored in the user folder. So consider a scenario, where I have 2 APIs one
is createStudent and the other is createUser.

Solution_Extracted:
But still, if you inject repository for one ent

# Save the summarized data to a new Excel file

In [ ]:
# Save the extracted issues solutions to a new Excel file
file_path = '/content/TextRank_Extracted_Issue_Solution.xlsx'

dataset.to_excel(file_path, index=False, engine='openpyxl')

# Evaluation of issue-solution pairs extracted by TextRank

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

def evaluate_summaries_at_sentence_level(df, ref_col, gen_col):
    precision_list = []
    recall_list = []
    f1_list = []

    for index, row in df.iterrows():
        ref_summary = row[ref_col]
        gen_summary = row[gen_col]

        if pd.isna(ref_summary) or pd.isna(gen_summary):
            continue

        ref_sentences = nltk.sent_tokenize(ref_summary)
        gen_sentences = nltk.sent_tokenize(gen_summary)

        ref_sentences_set = set(ref_sentences)
        gen_sentences_set = set(gen_sentences)

        precision = len(ref_sentences_set & gen_sentences_set) / len(gen_sentences_set) if gen_sentences_set else 0
        recall = len(ref_sentences_set & gen_sentences_set) / len(ref_sentences_set) if ref_sentences_set else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    metrics_df = pd.DataFrame({
        'precision': precision_list,
        'recall': recall_list,
        'f1': f1_list
    })

    return metrics_df

# Load the dataFrame containing both the extracted issue–solution pairs and the ground-truth data
df = pd.read_excel('TextRank_Extracted_Issue_Solution.xlsx')

question_metrics_df = evaluate_summaries_at_sentence_level(df, 'Ground_truth_Issue_Labeled', 'Issue_Extracted')
answer_metrics_df = evaluate_summaries_at_sentence_level(df, 'Ground_truth_Solution_Labeled', 'Solution_Extracted')

# Separate evaluation for question and answer summaries, retaining individual columns for comparison
question_metrics_df.columns = [f'Question_{col}' for col in question_metrics_df.columns]
answer_metrics_df.columns = [f'Answer_{col}' for col in answer_metrics_df.columns]

# Combine question and answer results into a single DataFrame
combined_metrics_df = pd.concat([question_metrics_df, answer_metrics_df], axis=1)

# Compute overall Precision, Recall, F1 scores (separately for questions and answers)
mean_question_metrics = question_metrics_df.mean()
mean_answer_metrics = answer_metrics_df.mean()

print("\nMean Precision, Recall, F1 Scores for \033[31mQuestions\033[0m:")
print(mean_question_metrics)

print("\nMean Precision, Recall, F1 Scores for \033[31mAnswers\033[0m:")
print(mean_answer_metrics)



Mean Precision, Recall, F1 Scores for Questions:
Question_precision    0.719490
Question_recall       0.640363
Question_f1           0.675396
dtype: float64

Mean Precision, Recall, F1 Scores for Answers:
Answer_precision    0.733834
Answer_recall       0.669997
Answer_f1           0.698194
dtype: float64
